In [59]:
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup as soup
from datetime import datetime as dt
from datetime import timedelta as td
import pandas as pd


# Read the old file from the share folder
file_name_offline = "/Users/Kumaran/Desktop/Sales.xlsx"
df_old_sales = pd.read_excel(file_name_offline)

# Log file
name_n_sales_offline = '/Users/Kumaran/Desktop/n_sales_offline.csv'


# Build up the start date as today and end date as one year from today
st_date = dt.strftime(dt.today(), '%Y-%m-%d')
ed_date = dt.strftime(dt.today() + td(days=365), '%Y-%m-%d')

url_template = "http://www.christies.com/Calendar?startdate={}&enddate={}".format(st_date,ed_date)
response = ureq(url_template)

contents = response.read()
parsed_html = soup(contents,"html.parser")
containers = parsed_html.findAll('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12 sale--items--item no-padding'})

# This function converts the string date to required format YYYYMMDD
def sale_date_parser(str_date):
    splt_date = str_date.split('-')
    if len(splt_date) < 2:                                              # When there is only start date
        lv_date = str_date
    else:                                                               # When there is both start date and end date
        lv_s_date = splt_date[0].strip()
        lv_date = splt_date[1].strip()

    sale_date_parser = dt.strptime(str(dt.today().year) + ' ' + lv_date, '%Y %d %b')

    lv_month = sale_date_parser.strftime('%m')
    lv_day = sale_date_parser.strftime('%d')

    if len(splt_date) < 2:
        parsed_st_date = str(sale_date_parser.year) + lv_month + lv_day
        parsed_ed_date = parsed_st_date
    else:
        parsed_ed_date = str(sale_date_parser.year) + lv_month + lv_day
        if len(lv_s_date) == 1:                                          #'1 - 12 Jan'
            st_dat = '0' + splt_date[0].strip()[:1]
        elif len(lv_s_date) > 2:                                         #'19 Feb - 1 Mar'
            st_dat1 = dt.strptime(str(dt.today().year) + ' ' + lv_s_date, '%Y %d %b')
            st_dat = str(st_dat1.strftime('%d'))
            lv_month = str(st_dat1.strftime('%m'))
        else:
            st_dat = splt_date[0].strip()[:2]                            #'16 - 24 Jan'
        parsed_st_date = str(sale_date_parser.year) + lv_month + st_dat
    return parsed_st_date,parsed_ed_date


# with open("/Users/Kumaran/Desktop/Christie's.csv", 'w') as file:
header = "SaleNumber,SaleText,SaleLocation,Sale_url,SaleDate,StartDate,EndDate,SaleID\n"
# file.write(header)

# df_header = ["SaleNumber", "SaleText", "SaleLocation", "Sale_url", "SaleDate", "StartDate", "EndDate", "SaleID"]
df_header = ["sale_no", "sale_loc", "url", "start", "end", "sale_id"]
df_incmg_sales = pd.DataFrame(columns=df_header)
incmg_list = []
tomw_list= []
tomorrow = (dt.today() + td(1)).strftime("%Y%m%d")

for container in containers:
    Sale_number = int(container.find('span',{'class':'image-description--sale-number hidden-xs hidden-sm p--primary_large'}).getText())
    # Sale_number = container.span.getText()[7:]
    Sale_text = container.h6.a.getText().replace(',',' ')
    Sale_location = container.find('span',{'class':'image-description--location hidden-xs hidden-sm p--primary_large'}).getText()
    Sale_date = container.h4.getText()
    Start_date, End_date = sale_date_parser(Sale_date)
    Sale_url = container.find('a',{'class':'image-hyperlink'}).get('href')
    s_indx = Sale_url.find('SaleID=')+7
    Sale_id = Sale_url[s_indx:s_indx+5]

    # incmg_list.append([Sale_number,Sale_text,Sale_location,Sale_url, Sale_date, Start_date, End_date, Sale_id])
    incmg_list.append([Sale_number, Sale_location, Sale_url, Start_date, End_date, Sale_id])

# Capture tomorrow's sale separately
    if Start_date == tomorrow:
        # tomw_list.append([Sale_number,Sale_text,Sale_location,Sale_url, Sale_date, Start_date, End_date, Sale_id])
        tomw_list.append([Sale_number, Sale_location, Sale_url, Start_date, End_date, Sale_id])


    # print(Sale_number, '|', Sale_text, '|', Sale_location, '|', Sale_url, '|', Sale_date, '|', Start_date, '|',End_date, '|', Sale_id)
    # file.write(Sale_number + ',' + Sale_text + ',' + Sale_location + ',' + Sale_url + ',' + Sale_date + ',' + Start_date + ',' + End_date + ',' + Sale_id + '\n')

# Append the incoming sales which was extracted
dff = pd.DataFrame(incmg_list,columns=df_header)                    # Append the output list to a temp DF
df_incmg_sales = pd.concat([dff])                                   # Append the temp to main DF
# df_new_sales.drop(['SaleText','SaleDate'],axis=1,inplace=True)

# capture tomorrow sales
dff = pd.DataFrame(tomw_list,columns=df_header)                    # Append the output list to a temp DF
df_tomorrow_sales = pd.concat([dff])                               # Append the temp to main DF
# df_tomorrow_sales.drop(['SaleText', 'SaleDate'], axis=1, inplace=True) # For now just drop these


# Check and update the new records from new dataframe onto the old dataframe
logs =[]
df_unique_new = pd.DataFrame(columns=df_header, index=range(1))
new_sales = pd.DataFrame(columns=df_header, index=range(1))

logs.append("Following New Sales are being Loaded")
for index, row in df_incmg_sales.iterrows():
# Only the sale that are not online.
    if row.sale_loc != 'Online':
# Check if the sale is already in the table
        if len(df_old_sales.loc[df_old_sales['sale_no'] == row.sale_no].index) == 0:
# if not add it.
            for a in range(1):
                df_unique_new.loc[a].sale_no  = row.sale_no
                df_unique_new.loc[a].sale_loc = row.sale_loc
                df_unique_new.loc[a].url      = row.url
                df_unique_new.loc[a].start    = row.start
                df_unique_new.loc[a].end      = row.end
                df_unique_new.loc[a].sale_id  = row.sale_id
                df_old_sales = df_old_sales.append(df_unique_new, ignore_index=True)
                logs.append(str(row.sale_no) + "-" + row.sale_loc)
                print(str(row.sale_no) + "-" + row.sale_loc)
# Also create a file with the new sales separately
                new_sales = new_sales.append(df_unique_new, ignore_index=True)

# Now dump the  dataframe data to 'Old sales list'
df_old_sales.to_excel(file_name_offline, header=True, index=False)

# Output the log file from the Dataframe
logs_out = pd.DataFrame(logs, columns=['log'])
logs_out.to_csv(name_n_sales_offline, header=False, index=False)